In [1]:
import pandas as pd
import numpy as np

from IPython.core.display import display, HTML
#display(HTML("<style>.jp-CodeCell.jp-mod-outputsScrolled .jp-Cell-outputArea { max-height: 40em; }</style>"))
import ipywidgets as widgets
from ipywidgets import interact, interactive, interact_manual

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

import os
os.chdir(r"C:\Users\randy\OneDrive\Desktop\School_project")
pd.set_option("display.max_rows", None, "display.max_columns", None)

import warnings
warnings.filterwarnings("ignore")

## Household Income
https://ofm.wa.gov/washington-data-research/economy-and-labor-force/median-household-income-estimates

In [2]:
income_df = pd.read_excel("median_household_income_estimates.xlsx",dtype = 'string')

In [3]:
features = [2020]
income_df = income_df.set_index('County')
income_df = income_df[features]
income_df.dropna(inplace=True, axis=0)
income_df.columns = ['Household_Income']
income_df.Household_Income = pd.to_numeric(income_df.Household_Income)
income_df.Household_Income = round(income_df.Household_Income )

income_mean = np.percentile(income_df['Household_Income'],35)
income_df = income_df[income_df['Household_Income'] >=income_mean]
income_df = income_df.Household_Income

## Housing Metrics
https://www.redfin.com/news/data-center/

In [4]:
#home_df = pd.read_csv(r"C:\Users\randy\OneDrive\Desktop\School_project\city_market_tracker.tsv000\city_market_tracker.tsv",sep='\t',header=0)
home_df = pd.read_csv("Washington_Home_Values.csv", dtype = 'string') 

In [5]:
Richland = ['Richland',412500.00,0,0,0]
Kennewick = ['Kennewick',387083.00,0,0,0]
Pasco = ['Pasco',387083.00,0,0,0]
WallaWalla = ['Walla Walla',377083.00,0,0,0]
WestRichland = ['West Richland',446000.00,0,0,0]
Pullman = ['Pullman',387083.00,0,0,0]
Prosser = ['Prosser',362000.00,0,0,0]
BentonCity = ['Benton City',368000.00,0,0,0]

missing_list = [Richland, Kennewick, Pasco, WallaWalla, WestRichland, Pullman, Prosser, BentonCity]


home_features = ['City','median_sale_price','median_sale_price_mom','median_dom']
data_range = '2021-01-01|2021-02-01|2021-03-01|2021-04-01|2021-05-01|2021-06-01|2021-07-01|2021-08-01|2021-09-01|2021-10-01'

home_df.rename(columns={'city' : 'City'}, inplace=True)  ## channge city to conform with other datasets
home_df = home_df[home_df.period_begin.str.contains(data_range)] ## Select latest date
home_df = home_df[home_df.property_type.str.contains('Single Family Residential')]

home_df.dropna(inplace=True)

## Convert numeric data types
home_df.median_sale_price = home_df.median_sale_price.astype(float)
home_df.median_sale_price_mom = home_df.median_sale_price_mom.astype(float)
home_df.median_ppsf= home_df.median_ppsf.astype(float)
home_df.median_dom= home_df.median_dom.astype(float)

## Group to find median of date range
median_p = home_df.groupby(['City'])['median_sale_price'].median()
median_p_mom = home_df.groupby(['City'])['median_sale_price_mom'].median()
median_ppsf = home_df.groupby(['City'])['median_ppsf'].median()
median_dom = home_df.groupby(['City'])['median_dom'].median()

## Combine The DataFrames
home_df = pd.concat([median_p,median_p_mom, median_ppsf, median_dom], axis = 1)
home_df = home_df.groupby(['City']).first()
home_df.reset_index(inplace = True)

## Append missing city values
home_df = home_df.append(pd.DataFrame(missing_list, columns=list(home_df)))

## Fill missing Values with median of columns
home_df = home_df.mask(home_df==0).fillna(home_df.median())

## Trim The Features
home_df = home_df[home_features]
home_df.sort_values(by=['median_sale_price'], ascending = False, inplace=True)
home_df.reset_index(inplace=True)


## School Directory
https://eds.ospi.k12.wa.us/DirectoryEDS.aspx

In [6]:
directory_df = pd.read_excel('Washington_Education_School_Directory_07_17_2020.xlsx',dtype= 'string')

In [7]:
d_features = ['LEACode', 'SchoolCode','SchoolName', 'City', 'GradeCategory']
directory_df = directory_df[d_features]
directory_df['ST_SCHID'] = "WA-"+ directory_df.LEACode + "-" + directory_df.SchoolCode
d_features = ['ST_SCHID','GradeCategory','City','SchoolCode']
dir_df = directory_df[d_features]

## Student Growth
https://data.wa.gov/Education/Report-Card-Growth-for-2018-19/uj4q-wr8d

In [8]:
growth_df = pd.read_csv("Report_Card_Growth_for_2018-19.csv", dtype ='string')

In [9]:
growth_features = ['County','SchoolCode','SchoolName','CurrentSchoolType', 'GradeLevel' , 'PercentHighGrowth', 'PercentLowGrowth']
growth_df = growth_df[growth_features]
growth_df = growth_df[~growth_df.SchoolName.str.contains('State Total')]
growth_df = growth_df[growth_df.GradeLevel.str.contains('1|2|3|4|5')]
growth_df.PercentHighGrowth = pd.to_numeric(growth_df.PercentHighGrowth)
growth_df.PercentLowGrowth = pd.to_numeric(growth_df.PercentLowGrowth)
growth_df.dropna(inplace = True)

## Isolate and merge the median p_high and p_low values from each school
df_high = growth_df.groupby(['SchoolCode'])['PercentHighGrowth'].median()
df_low = growth_df.groupby(['SchoolCode'])['PercentLowGrowth'].median()
df_m = pd.merge(df_high,df_low, on='SchoolCode', how='inner')

## Create the 'Overall' feature
df_m['Test_Growth'] = df_m.PercentHighGrowth - df_m.PercentLowGrowth

## Merge with df_m to grab median and overall values
growth_df = pd.merge(df_m,growth_df, on='SchoolCode', how='inner')

## Merge with directory for standardized school code
growth_df = pd.merge(growth_df,dir_df,on='SchoolCode', how = 'inner')

## Take First
growth_df = growth_df.groupby(['ST_SCHID']).first().reset_index()

## Reduce Features
growth_features = ['SchoolName','County','Test_Growth','ST_SCHID']
growth_df = growth_df[growth_features]
growth_df.sort_values(by = ['Test_Growth'], ascending = False,  inplace=True)


## Student Test Scores
https://www2.ed.gov/about/inits/ed/edfacts/data-files/index.html


In [10]:
state = 'WASHINGTON'
student_type = 'ALL'
grade = '00'

#### Math

In [11]:
math_df = pd.read_csv('EDF_SCH_AP_MTH_1819_PUBL.csv', dtype='string')

In [12]:
math_features = ['ST_SCHID', 'PCTPROF']
math_df = math_df = math_df[math_df.STNAM.str.contains(state)]
math_df = math_df = math_df[math_df.CATEGORY.str.contains(student_type)]
math_df = math_df = math_df[math_df.GRADE.str.contains(grade)]
math_df.PCTPROF = math_df.PCTPROF.str[-2:] # Slice last two characters

math_df.PCTPROF = math_df.PCTPROF.str.replace('PS', '1')
math_df.PCTPROF = math_df.PCTPROF.str.replace('E5', '1')
math_df.PCTPROF = pd.to_numeric(math_df.PCTPROF)
math_df = math_df[math_features]
math_df.rename({'PCTPROF' : "Math_Proficiency"},axis=1, inplace=True)

#### Reading

In [13]:
reading_df = pd.read_csv('EDF_SCH_AP_RLA_1819_PUBL.csv', dtype='string')

In [14]:
reading_features = ['ST_SCHID', 'PCTPROF']
reading_df = reading_df = reading_df[reading_df.STNAM.str.contains(state)]
reading_df = reading_df = reading_df[reading_df.CATEGORY.str.contains(student_type)]
reading_df = reading_df = reading_df[reading_df.GRADE.str.contains(grade)]
reading_df.PCTPROF = reading_df.PCTPROF.str[-2:] # Slice last two characters

reading_df.PCTPROF = reading_df.PCTPROF.str.replace('PS', '1')
reading_df.PCTPROF = reading_df.PCTPROF.str.replace('E5', '1')
reading_df.PCTPROF = pd.to_numeric(reading_df.PCTPROF)
reading_df = reading_df[reading_features]
reading_df.rename({'PCTPROF' : "Reading_Proficiency"},axis=1, inplace=True)

#### Math/Reading/Growth-Merged

In [15]:
test_scores_df = pd.merge(math_df,reading_df, on='ST_SCHID', how='inner')
test_scores_df = pd.merge(dir_df, test_scores_df,on='ST_SCHID', how='inner') # Add directory info
test_scores_df = pd.merge(test_scores_df, growth_df, on='ST_SCHID', how='inner') # Add Growth info
test_scores_df['Test_Scores'] = (test_scores_df.Math_Proficiency + test_scores_df.Reading_Proficiency) / 2


## Crime/Population
https://ucr.fbi.gov/crime-in-the-u.s/2019/crime-in-the-u.s.-2019/tables/table-8/table-8-state-cuts/washington.xls

In [16]:
crime_df = pd.read_excel("washington_crime.xls",dtype = 'string')

In [17]:
crime_df.columns = crime_df.iloc[3]  ## Move Column Headers Up
crime_df = crime_df.iloc[4: , :]  ## Shift Rows Up
crime_df.City = crime_df.City.str.upper()
crime_df = crime_df.set_index('City') ## Set Index
crime_df = crime_df.astype(float) ## Convert Data Types
crime_df.rename({'Murder and\nnonnegligent\nmanslaughter': 'Murder'},axis=1, inplace=True)

population_df = crime_df.Population ## Create Population DF
crime_df.drop(['Population'],axis = 1, inplace=True) ## Temporarily drop Population
crime_df['total_crime'] = crime_df.sum(axis=1)  ## Sum Values W/out Population 
crime_df = pd.merge(crime_df,population_df, on='City', how= 'inner') ## Recombine population
crime_df['Crime_Per_1000'] = crime_df.total_crime / (crime_df.Population/1000)  ## Create New Feature
crime_df.dropna(inplace = True)
crime_df.sort_values( by = 'Crime_Per_1000', inplace=True)
crime_df.reset_index(inplace=True)


## Merged/Sorted DataFrames

In [18]:
df = test_scores_df
df.City = df.City.str.upper()
home_df.City = home_df.City.str.upper()

## Merge Dataframes Together
#df = pd.merge(df,hv_m, on='County', how='inner')
df = pd.merge(df,home_df, on='City', how='inner')
df = pd.merge(df, income_df, on='County', how='inner')
df = pd.merge(df, crime_df, on='City', how ='inner')

df.drop('GradeCategory', axis=1, inplace = True, errors = 'ignore')
df.dropna(inplace=True)

# <span style="color:blue;"> Metric Weighting For Final Sort </span>

In [19]:

df_features = ['SchoolName','City','County','Test_Scores','Test_Growth','median_sale_price',
               'median_sale_price_mom','Household_Income','Population','Crime_Per_1000','Combined_Values']

## Create a big picture metric
scaled_df = df
scaled_df[['Test__Growth','Test__Scores', 'median__sale_price_mom','median__sale_price', 'Household__Income', 'Crime__Percent']] = \
scaler.fit_transform(df[['Test_Growth','Test_Scores', 'median_sale_price_mom','median_sale_price', 'Household_Income', 'Crime_Per_1000']])


print('Click and drag each slider to desired location based on preference towards the given metric.')
print('A value of "0" will remove the variable as a factor, a value of "3" places the variable as most important.')
print('')
print ('Test_Growth:   = The year over year gains within standardized testing results.')
print ('Test_Scores:   = The combined results from within the standardized reading and math test scores.')
print ('Investment:    = The home value appreciation of a given city.')
print ('Income:        = The average household income from within the county that the school is located within.')
print ('Home_Value:    = The median home value from within the given city.  A higher input value equates to a preference for lower priced homes')
print ('Crime:         = The total crime as a percent of total population from within the given city.')
print ('Minimum:       = The minimum school test score to accept from within the results.')
print('')
print('County:         = A manual text box that can be used to view results at a County level')
print('City:           = A manual text box that can be used to view results at a City level')
print('')

@interact
def f( Test_Growth = (0,3), Test_Scores=(0,3), Investment = (0,3),Income=(0,3), Home_Price=(0,3), Crime=(0,3), Minimum = (0,95), County = '', City = ''):
    
    
    df['Combined_Values'] = (scaled_df.Test__Growth * Test_Growth) + \
                            (scaled_df.Test__Scores * Test_Scores) + \
                            (scaled_df.median__sale_price_mom * Investment) + \
                            (scaled_df.Household__Income * Income) - \
                            (scaled_df.median__sale_price * Home_Price) - \
                            (scaled_df.Crime__Percent * Crime) 
    def dataframe(df = df):
        d = df[df_features]
        return (d)
    d = dataframe(df)
    d.sort_values(by = 'Combined_Values', ascending = False, inplace = True)
    d.reset_index(inplace=True)
    
    print("")
    if County and not City:
        d = d[(d['Test_Scores'] >=Minimum) & (d['County'].str.lower() == County.lower())]
    elif City and not County:
        d = d[(d['Test_Scores'] >=Minimum) & (d['City'].str.lower() == City.lower())]
    elif County and City:
        d = d[(d['Test_Scores'] >=Minimum) & (d['County'].str.lower() == County.lower()) & (d['City'].str.lower() == City.lower())]
    else:
        d = d[(d['Test_Scores'] >=Minimum)]    
    return (d)
    
 

Click and drag each slider to desired location based on preference towards the given metric.
A value of "0" will remove the variable as a factor, a value of "3" places the variable as most important.

Test_Growth:   = The year over year gains within standardized testing results.
Test_Scores:   = The combined results from within the standardized reading and math test scores.
Investment:    = The home value appreciation of a given city.
Income:        = The average household income from within the county that the school is located within.
Home_Value:    = The median home value from within the given city.  A higher input value equates to a preference for lower priced homes
Crime:         = The total crime as a percent of total population from within the given city.
Minimum:       = The minimum school test score to accept from within the results.

County:         = A manual text box that can be used to view results at a County level
City:           = A manual text box that can be used to v

interactive(children=(IntSlider(value=1, description='Test_Growth', max=3), IntSlider(value=1, description='Te…

### Housing Metrics

In [20]:
@interact
def f(City = '',Show_All = False, ):   
    d = home_df
    if City:
        d =(d.loc[d['City'].str.lower() == City.lower()])
    elif Show_All == True:
        d = d
    else:
        d = "Select Show_All or an individual City name to see more data"
    return (d)

interactive(children=(Text(value='', description='City'), Checkbox(value=False, description='Show_All'), Outpu…

### Crime Metrics

In [21]:
@interact
def f(City = '',Show_All = False, ):   
    d = crime_df
    if City:
        d =(d.loc[d['City'].str.lower() == City.lower()])
    elif Show_All == True:
        d = d
    else:
        d = "Select Show_All or an individual City name to see more data"
    return (d)

interactive(children=(Text(value='', description='City'), Checkbox(value=False, description='Show_All'), Outpu…

### Student Test Score Metrics

In [22]:
@interact
def f(City = '',Show_All = False, ):   
    d = test_scores_df
    d.sort_values(by = 'Test_Scores',ascending = False, inplace=True)
    #d.reset_index(inplace=True)
    if City:
        d =(d.loc[d['City'].str.lower() == City.lower()])
    elif Show_All == True:
        d=d
    else:
        d = "Select Show_All or an individual City name to see more data"
    return (d)

interactive(children=(Text(value='', description='City'), Checkbox(value=False, description='Show_All'), Outpu…